<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Assesment-2" data-toc-modified-id="Assesment-2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Assesment 2</a></span><ul class="toc-item"><li><span><a href="#Get-data" data-toc-modified-id="Get-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get data</a></span></li></ul></li><li><span><a href="#Question-1" data-toc-modified-id="Question-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Question 1</a></span></li><li><span><a href="#Question-4" data-toc-modified-id="Question-4-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Question 4</a></span><ul class="toc-item"><li><span><a href="#Inverted-Index" data-toc-modified-id="Inverted-Index-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Inverted Index</a></span></li></ul></li><li><span><a href="#Question-2" data-toc-modified-id="Question-2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Question 2</a></span><ul class="toc-item"><li><span><a href="#Elias-gamma" data-toc-modified-id="Elias-gamma-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Elias gamma</a></span></li><li><span><a href="#Elias-delta" data-toc-modified-id="Elias-delta-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Elias delta</a></span><ul class="toc-item"><li><span><a href="#Encoding" data-toc-modified-id="Encoding-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Encoding</a></span></li><li><span><a href="#Decoding" data-toc-modified-id="Decoding-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Decoding</a></span></li></ul></li><li><span><a href="#Golomb-and-Golomb-Rice" data-toc-modified-id="Golomb-and-Golomb-Rice-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Golomb and Golomb Rice</a></span></li></ul></li><li><span><a href="#Question-3" data-toc-modified-id="Question-3-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Question 3</a></span><ul class="toc-item"><li><span><a href="#Trie" data-toc-modified-id="Trie-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Trie</a></span></li><li><span><a href="#Predictive" data-toc-modified-id="Predictive-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Predictive</a></span></li><li><span><a href="#Auto-Correct" data-toc-modified-id="Auto-Correct-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Auto Correct</a></span></li></ul></li><li><span><a href="#Playground" data-toc-modified-id="Playground-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Playground</a></span></li></ul></div>

# Assesment 2
- [x] Write a program to show the implementation of a topical cralwer. Develop the crawler
program to handle various challenges (such as Parsing, Stemming, Lemmitization, 
Link Extraction, Canonicalization, Spider Trap etc.) faced by crawler while 
implementing. (Concurrent crawler of 3 threads)
- 2. Write a program to show the implementation of Elias Delta and Golomb Encoding￾decoding (both for m = 2N
and M ≠ 2N
) techniques. (x=37, b=11, b=8)
- [x] Write a program to extract the contents (excluding any tags) from two websites 
(https://en.wikipedia.org/wiki/Web_mining&https://en.wikipedia.org/wiki/Data_mining) 
and save the content in two separate .doc file. Construct a trie based on the content 
retrieved in Q. No. 3 using HashMap. Write a program to show the implementation of
Predictive Typing and Auto-Correct using the trie prepared.
- [x] Based on the contents of those two documents retrieved in Q. No. 3, prepare one 
inverted index (with proper representation) and save it in another text file. 

## Get data

In [77]:
from collections import deque
from bs4 import BeautifulSoup
import sys
import urllib
from html.parser import HTMLParser
from urllib.request import urlopen
from urllib import parse
import requests    
import re    
from urllib.parse import urlparse 
import re
import numpy as np
from multiprocessing import Pool
import os
from multiprocessing import Process

In [3]:
url = "https://en.wikipedia.org/wiki/Web_mining"

urlf = urlopen(url)
soup = BeautifulSoup(urlf.read(),"lxml")
with open("datawm.txt","w+") as f:
    for a in soup.find_all('p'):
        f.write(a.getText()+"\n")
        
url2 = "https://en.wikipedia.org/wiki/Data_mining"
urlf = urlopen(url2)
soup = BeautifulSoup(urlf.read(),"lxml")
with open("datadm.txt","w+") as f:
    for a in soup.find_all('p'):
        f.write(a.getText()+"\n")


In [22]:
datadm = open("datadm.txt","r").read()
datawm = open("datawm.txt","r").read()

# Question 1

In [98]:

class topicCrawler():    
    def __init__(self, starting_url,depth):    
        self.starting_url = starting_url    
        self.visited = set()  
        self.depth = depth

    def get_html(self, url):    
        try:    
            html = requests.get(url)    
        except Exception as e:    
            print(e)    
            return ""    
        return html.content.decode('latin-1')    

    def get_links(self, url):    
        html = self.get_html(url)    
        parsed = urlparse(url)    
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html)    
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base       

        return set(filter(lambda x: 'mailto' not in x, links))    

    def extract_info(self, url):                                
        html = self.get_html(url)                               
        return None

    def avoid_traps(self, x):
        # Checks for = files, weird URLs, long ones, depth of searching, repeats
        flag = 0
        files = ['.jpg','.png','.pdf']
        if len(x)>100:
            flag =1
        if any([('?' in x),('%' in x),('#' in x),('&' in x)]):
            flag =1
        if np.sum([1 for a in files if a in x])>0:
            flag=1
        x = x.replace('//',"")
        splii = x.split('/')
        if len(splii)>self.depth+1:
            flag=1
        if len(set(splii))!=len(splii):
            flag=1
        
        
        return flag
    
    def remove_traps(self,x):
        temp = []
        for a in x:
            if self.avoid_traps(a)!=1:
                temp.append(a)
        return temp

    def crawl(self, url):     
#         print(os.getppid())
        print(threading.current_thread())
        self.frontier = self.get_links(url)
        
        self.cleaned = self.remove_traps(self.frontier)
        print('Tot:',len(self.frontier),'Cleaned:',len(self.cleaned))
        print(self.cleaned)
        
        
        for link in self.get_links(url):    
            if link in self.visited:        
                continue                    
            print(link)                 
            self.visited.add(link)            
            info = self.extract_info(link)    
            self.crawl(link)                  

    def start(self):                     
        self.crawl(self.starting_url)    

In [85]:
from concurrent.futures import ThreadPoolExecutor
import threading

In [100]:

executor = ThreadPoolExecutor(max_workers=3)
task1 = executor.submit(topicCrawler('https://en.wikipedia.org/wiki/Data_mining',depth=2).start())
task2 = executor.submit(topicCrawler('https://en.wikipedia.org/wiki/Web_mining',depth=2).start())
task2 = executor.submit(topicCrawler('https://en.wikipedia.org/wiki/Text_mining',depth=2).start())


<_MainThread(MainThread, started 140051483219776)>
Tot: 841 Cleaned: 572
['https://en.wikipedia.org/wiki/Artificial_intelligence', 'https://en.wikipedia.org/wiki/Review_of_Economic_Studies', 'https://en.wikipedia.org/wiki/Autoencoder', 'https://en.wikipedia.org/wiki/Data_reduction', 'https://en.wikipedia.org/wiki/Multilayer_perceptron', 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'https://en.wikipedia.org/wiki/OCLC', 'https://en.wikipedia.org/wiki/Information_system', 'https://en.wikipedia.org/wiki/Structured_prediction', 'https://en.wikipedia.org/wiki/Knowledge_representation_and_reasoning', 'https://en.wikipedia.org/wiki/Template:Data_warehouses', 'https://en.wikipedia.org/wiki/Software_deployment', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Embedded_system', 'https://en.wikipedia.org/wiki/Data_recovery', 'https://en.wikipedia.org/wiki/Virtual_machine', 'https://en.wikipedia.org/wiki/Computability_theory', 'https://en.wikipedia.or

Tot: 240 Cleaned: 127
['https://en.wikipedia.org/wiki/Data_clustering', 'https://en.wikipedia.org/wiki/Artificial_intelligence', 'https://en.wikipedia.org/wiki/Wikipedia:Citing_sources', 'https://en.wikipedia.org/wiki/Web_scraping', 'https://en.wikipedia.org/wiki/Vector_space_model', 'https://en.wikipedia.org/wiki/Information_management', 'https://en.wikipedia.org/wiki/Autonomous_agent', 'https://foundation.wikimedia.org/wiki/Cookie_statement', 'https://en.wikipedia.org/wiki/Category:All_articles_lacking_in-text_citations', 'https://foundation.wikimedia.org/wiki/Privacy_policy', 'https://en.wikipedia.org/wiki/Semi-structured_data', 'https://en.wikipedia.org/wiki/GB_18030', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Help:Maintenance_template_removal', 'https://en.wikipedia.org/wiki/Personal_data', 'https://en.wikipedia.org/wiki/Natural_language_processing', 'https://en.wikipedia.org/wiki/Personalization', 'http://www.cs.uic.edu/~liub/WebMiningBook.html

# Question 4

## Inverted Index

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

stop_words = set(stopwords.words('english'))


In [24]:
def return_list_indices(word, lis):
    temp = []
    for i in range(len(lis)):
        if lis[i]== word:
            temp.append(i)
    return temp
    

In [25]:
def doc_index(x,docname):
    data_rem_stop = word_tokenize(x)
    data_rem_stop = [x for x in data_rem_stop if x not in stop_words]
    data_rem_stop = [x.lower() for x in data_rem_stop if all([len(x)>1,x.isalpha()])]
    counts= Counter(data_rem_stop)
    temp_dic = {}
    for a in set(data_rem_stop):
        temp_dic[a] = f"<{docname},{counts[a]},{return_list_indices(a,data_rem_stop)}>"
#         print(f"<{docname},{counts[a]},{return_list_indices(a,data_rem_stop)}>")
#         break
    return temp_dic
    


In [26]:
dict1 = doc_index(datadm,'id1')

In [27]:
dict2 = doc_index(datawm,'id2')

In [28]:
all_words = list(dict1.keys())+list(dict2.keys());all_words

['inferences', 'analyze', 'campaign', 'semma', 'preparation', 'actually', 'international', 'explorations', 'korea', 'consumers', 'associated', 'refer', 'records', 'congress', 'uncovers', 'more', 'combining', 'sets', 'might', 'jdm', 'property', 'conference', 'principles', 'defined', 'see', 'business', 'curves', 'israel', 'topic', 'raise', 'involves', 'metrics', 'case', 'proprietary', 'removes', 'violation', 'works', 'titled', 'reported', 'knowledge', 'kdd', 'analytics', 'limit', 'uses', 'artificial', 'fishing', 'open', 'rights', 'remaining', 'using', 'icde', 'play', 'give', 'modified', 'compiled', 'reach', 'misnomer', 'tools', 'complexity', 'visualization', 'overcome', 'usually', 'one', 'standard', 'necessary', 'fair', 'referring', 'notable', 'informed', 'new', 'anonymity', 'uncover', 'meet', 'markup', 'selling', 'online', 'personally', 'investigating', 'validation', 'controlled', 'conditions', 'mostly', 'version', 'contexts', 'input', 'forums', 'overridden', 'summary', 'uncovering', 'c

In [29]:
inverted_index = {**doc_index(datadm,'id1'),**doc_index(datawm,'id2')}

In [30]:
inverted_index['data']

'<id2,34,[3, 25, 152, 160, 168, 186, 194, 368, 403, 435, 440, 469, 474, 487, 494, 501, 512, 536, 589, 594, 598, 620, 638, 660, 723, 730, 768, 769, 795, 902, 906, 929, 933, 975]>'

In [31]:
for key, value in inverted_index.items():
    if key in dict1 and key in dict2:
        inverted_index[key] = str(value) +","+ str(dict1[key])

In [32]:
inverted_index

{'inferences': '<id1,1,[301]>', 'analyze': '<id2,1,[628]>,<id1,1,[724]>', 'campaign': '<id1,1,[258]>', 'semma': '<id1,2,[661, 681]>', 'preparation': '<id1,3,[228, 1137, 1177]>', 'actually': '<id1,2,[697, 757]>', 'international': '<id1,4,[449, 485, 945, 975]>', 'explorations': '<id1,2,[482, 956]>', 'korea': '<id1,1,[1566]>', 'consumers': '<id1,1,[1299]>', 'associated': '<id2,1,[360]>,<id1,1,[1091]>', 'refer': '<id1,2,[286, 327]>', 'records': '<id1,2,[169, 173]>', 'congress': '<id1,1,[1343]>', 'uncovers': '<id1,1,[1138]>', 'more': '<id2,1,[318]>,<id1,1,[1385]>', 'combining': '<id2,1,[616]>,<id1,1,[1153]>', 'sets': '<id1,7,[7, 542, 587, 617, 727, 1119, 1227]>', 'might': '<id2,6,[300, 406, 472, 540, 554, 605]>,<id1,2,[211, 1163]>', 'jdm': '<id1,3,[1000, 1004, 1008]>', 'property': '<id1,1,[1459]>', 'conference': '<id1,7,[450, 486, 491, 946, 972, 974, 976]>', 'principles': '<id1,1,[1303]>', 'defined': '<id2,1,[452]>,<id1,1,[625]>', 'see': '<id1,1,[1631]>', 'business': '<id1,5,[108, 430, 1054

In [33]:
with open('inverted_index.txt','w+') as f:
    for a in inverted_index:
#         temp_l = inverted_index[a]
        f.write(f"{a}, {inverted_index[a]}\n")

In [34]:
!cat inverted_index.txt

inferences, <id1,1,[301]>
analyze, <id2,1,[628]>,<id1,1,[724]>
campaign, <id1,1,[258]>
semma, <id1,2,[661, 681]>
preparation, <id1,3,[228, 1137, 1177]>
actually, <id1,2,[697, 757]>
international, <id1,4,[449, 485, 945, 975]>
explorations, <id1,2,[482, 956]>
korea, <id1,1,[1566]>
consumers, <id1,1,[1299]>
associated, <id2,1,[360]>,<id1,1,[1091]>
refer, <id1,2,[286, 327]>
records, <id1,2,[169, 173]>
congress, <id1,1,[1343]>
uncovers, <id1,1,[1138]>
more, <id2,1,[318]>,<id1,1,[1385]>
combining, <id2,1,[616]>,<id1,1,[1153]>
sets, <id1,7,[7, 542, 587, 617, 727, 1119, 1227]>
might, <id2,6,[300, 406, 472, 540, 554, 605]>,<id1,2,[211, 1163]>
jdm, <id1,3,[1000, 1004, 1008]>
property, <id1,1,[1459]>
conference, <id1,7,[450, 486, 491, 946, 972, 974, 976]>
principles, <id1,1,[1303]>
defined, <id2,1,[452]>,<id1,1,[625]>
see, <id1,1,[1631]>
business, <id1,5,[108, 430, 1054, 1079, 1371]>
curves, <id1,1,[896]>
israel, <id1,1,[1563]>
topic, <id1,1,[416]>
raise, <id1,1,[1106]>
involves, <id2,1,[737]>,<i

# Question 2

## Elias gamma

In [35]:
def elias_gamma(x):
    temp = bin(x)[2::]; temp
    return ''.join(['0' for i in range(len(temp)-1)])+temp

In [36]:
elias_gamma(37)

'00000100101'

## Elias delta

In [37]:
import numpy as np

### Encoding

In [38]:
def elias_delta_enc(x):
    temp = 1+np.floor(np.log2(x))
    print(temp)
    gam = elias_gamma(int(temp))
    print(gam)
    binrep = bin(x)[3:]
    print(gam+binrep)
    return gam+binrep
    

In [39]:
elias_delta_enc(9)

4.0
00100
00100001


'00100001'

### Decoding

In [40]:
def elias_delta_dec(x):
    l = x.find('1');l
    print(l)
    m = 2*l
    print('1'+x[m+1::])
    return int('1'+x[m+1::],2)

In [41]:
elias_delta_dec('00100011')

2
1011


11

In [42]:
elias_delta_dec(elias_delta_enc(34))

6.0
00110
0011000010
2
100010


34

## Golomb and Golomb Rice

In [43]:
# (x=37, b=11, b=8)

In [1]:
def unary(x):
    return ''.join(['0' for i in range(x)])+'1'

In [125]:
def binary(x):
    return bin(x)

In [2]:
unary(5)

'000001'

In [121]:
def golomb(s,m):
    k= int(np.ceil(np.log2(m)))
    r = s%m
    t = 2**k-m
    un = unary(int(np.floor(8/7)))
    print(r,t)
    if r<=t:
        k = k-1
    else:
        r = r+t
    print(r)
    bi = binary(r)
    return un,bi
    

In [122]:
golomb(8,7)

1 1
1


('01', '0b1')

# Question 3

In [50]:
def letter_chunck(x):
    temp = []
    for a in x:
        temp.append(a[0])
    return set(temp)

In [51]:
lc = letter_chunck(inverted_index);lc

{'v', 's', 'h', 'd', 'e', 'f', 'g', 'j', 'c', 'w', 'n', 'r', 'y', 't', 'o', 'a', 'q', 'i', 'p', 'b', 'm', 'l', 'k', 'u'}

In [52]:
hmap_trie = {}

In [53]:
for a in lc:
    hmap_trie[a] = {}
    temp_letter= {}
    for b in inverted_index:
        if b[0] == a:
            temp_letter[b]=inverted_index[b]
    hmap_trie[a] = temp_letter

## Trie

In [54]:
hmap_trie

{'v': {'violation': '<id1,1,[1273]>', 'visualization': '<id1,1,[65]>', 'validation': '<id1,1,[645]>', 'version': '<id1,1,[779]>', 'violates': '<id2,1,[479]>,<id1,1,[1256]>', 'validity': '<id2,2,[366, 931]>,<id1,1,[303]>', 'variety': '<id1,1,[355]>', 'very': '<id1,1,[977]>', 'volume': '<id1,1,[275]>', 'vector': '<id2,2,[834, 843]>,<id1,1,[574]>', 'various': '<id1,1,[1157]>', 'valid': '<id1,1,[818]>', 'variations': '<id1,1,[631]>', 'verify': '<id1,1,[801]>', 'viewed': '<id1,2,[1567, 1576]>', 'via': '<id1,1,[1344]>', 'vista': '<id2,1,[692]>', 'verification': '<id2,1,[932]>', 'volumes': '<id2,1,[243]>', 'view': '<id2,5,[757, 760, 762, 772, 810]>'}, 's': {'semma': '<id1,2,[661, 681]>', 'sets': '<id1,7,[7, 542, 587, 617, 727, 1119, 1227]>', 'see': '<id1,1,[1631]>', 'standard': '<id2,1,[968]>,<id1,5,[633, 658, 992, 999, 1023]>', 'selling': '<id2,1,[485]>,<id1,1,[1279]>', 'summary': '<id2,1,[52]>,<id1,1,[195]>', 'submissions': '<id1,1,[498]>', 'storage': '<id1,1,[537]>', 'sufficient': '<id1,1,

In [55]:
hmap_trie['h']

{'happening': '<id1,1,[794]>', 'hypothesis': '<id1,2,[334, 776]>', 'however': '<id2,1,[611]>,<id1,8,[308, 424, 629, 662, 1056, 1223, 1300, 1490]>', 'hipaa': '<id1,4,[1353, 1355, 1374, 1409]>', 'hnc': '<id1,1,[383]>', 'harbor': '<id1,1,[1302]>', 'hypotheses': '<id1,3,[253, 312, 772]>', 'high': '<id2,1,[585]>,<id1,1,[1052]>', 'hidden': '<id1,2,[272, 583]>', 'health': '<id1,1,[1348]>', 'harm': '<id1,1,[1266]>', 'highest': '<id1,1,[489]>', 'hosted': '<id1,1,[943]>', 'histories': '<id1,1,[1244]>', 'harvesting': '<id1,1,[402]>', 'hargreaves': '<id1,1,[1467]>', 'hyperlinks': '<id2,2,[71, 75]>', 'heterogeneity': '<id2,1,[667]>', 'hyponymy': '<id2,1,[215]>', 'hz': '<id2,1,[953]>', 'hard': '<id2,1,[559]>', 'hyperlink': '<id2,1,[803]>', 'hypertext': '<id2,1,[677]>', 'html': '<id2,2,[798, 969]>', 'higher': '<id2,2,[241, 727]>'}

## Predictive

In [56]:
def make_trie(words):
    root = dict()
    for word in words:
        current_dict = root
        for letter in word:
#             print(letter)
            current_dict = current_dict.setdefault(letter, {})
        current_dict['listWords'] = word
    return root

In [103]:
d_n = make_trie(list(set(all_words)));d_n

{'i': {'n': {'f': {'e': {'r': {'e': {'n': {'c': {'e': {'s': {'listWords': 'inferences'}, 'listWords': 'inference'}}}}, 'r': {'e': {'d': {'listWords': 'inferred'}}}, 'listWords': 'infer'}}, 'o': {'r': {'m': {'e': {'d': {'listWords': 'informed'}}, 'a': {'t': {'i': {'o': {'n': {'listWords': 'information'}}}}}}}}}, 't': {'e': {'r': {'n': {'a': {'t': {'i': {'o': {'n': {'a': {'l': {'listWords': 'international'}}}}}}}, 'e': {'t': {'listWords': 'internet'}}}, 'p': {'r': {'e': {'t': {'listWords': 'interpret', 'i': {'n': {'g': {'listWords': 'interpreting'}}}, 'a': {'t': {'i': {'o': {'n': {'listWords': 'interpretation'}}}}}}}}}, 'c': {'h': {'a': {'n': {'g': {'e': {'a': {'b': {'l': {'y': {'listWords': 'interchangeably'}}}}}}}}}}, 'e': {'s': {'t': {'i': {'n': {'g': {'listWords': 'interesting', 'n': {'e': {'s': {'s': {'listWords': 'interestingness'}}}}}}}, 'listWords': 'interest', 's': {'listWords': 'interests'}}}}, 's': {'e': {'c': {'t': {'i': {'o': {'n': {'listWords': 'intersection'}}}}}}}, 'd': {

In [108]:
import re
def fix_return(x):
    x = str(x)
    w1 = re.findall("{'listWords[a-z: ']*'}",x)
    temp_wor = []
    for a in w1:
        w2 = re.findall(" '[a-z]*'",a.replace("![a-z]",""))
        w2 = ''.join(e for e in w2[0] if e.isalnum())
        temp_wor.append(w2)

    return temp_wor

In [109]:
temp = d_n
for i in range(10):
    x = input()
    try:
        temp = temp[x]
        print(fix_return(temp))
    except KeyError:
        print('Word not in index, try again')
        break

s
['space', 'spatial', 'spam', 'sponsorship', 'specially', 'specific', 'semma', 'semantic', 'sets', 'settlement', 'seen', 'service', 'servers', 'serious', 'second', 'security', 'selling', 'selection', 'sequential', 'several', 'separate', 'search', 'sexual', 'standards', 'started', 'states', 'statistics', 'statistical', 'statisticians', 'stages', 'stalled', 'stakeholder', 'storage', 'stored', 'strengthen', 'structures', 'structural', 'strong', 'steps', 'studies', 'summary', 'summarized', 'submissions', 'subjective', 'subsequently', 'subset', 'subspace', 'subfield', 'sufficient', 'suggests', 'surveillance', 'supported', 'supplant', 'successors', 'score', 'schema', 'schemes', 'science', 'scale', 'since', 'single', 'sites', 'situation', 'similarity', 'similarly', 'simple', 'simplified', 'six', 'sigmod', 'sigkdd', 'significant', 'size', 'systems', 'sophisticated', 'solution', 'society', 'software', 'sources', 'south', 'some', 'says', 'save', 'safe', 'santos', 'sample', 'snowden', 'snooping'

## Auto Correct

In [129]:
def retu_closest(s):
    temp = d_n
    for i in range(len(s)):
        x = s[i]
        try:
            temp = temp[x]
#             print(fix_return(temp))
        except KeyError:
            break
    return fix_return(temp)

In [147]:
def word_sim(s1,s2):
    if len(s1)==len(s2):
        temp = set(list(s1)).intersection(set(list(s2)))
        return len(temp)
    else:
        return 0

In [171]:
def corrector(s):
    wds = retu_closest(s)
#     print(wds)
    wd, c = s,0
    for a in wds:
        sim = word_sim(wd, a)
#         print(sim,wd, a)
        if sim!=0:
            if sim>c:
                wd =a
                c = sim
    
    print(wd)

In [174]:
s = 'santop'
# 'some', 'says', 'save', 'safe', 'santos', 'sample',
corrector(s)

santos


# Playground

In [3]:
l_temp = ["spider","speech","spdemergency"]

In [4]:
l_temp.sort();l_temp

['spdemergency', 'speech', 'spider']

In [5]:
def make_trie(words):
    root = dict()
    for word in words:
        current_dict = root
        for letter in word:
#             print(letter)
            current_dict = current_dict.setdefault(letter, {})
        current_dict['listWords'] = word
    return root

In [6]:
%pprint

Pretty printing has been turned OFF


In [7]:
import pprint

In [8]:
d_n = make_trie(l_temp)

In [9]:
d_n['s']['p']['i']

{'d': {'e': {'r': {'listWords': 'spider'}}}}

In [18]:
l_te = []
return_list_set(d_n['s']['p']['i'])

4


In [19]:
d_n['s']['p']['i']

{'d': {'e': {'r': {'listWords': 'spider'}}}}

In [100]:
temp = d_n
for i in range(3):
    x = input()
    try:
        temp = temp[x]
        print(temp)
    except KeyError:
        print('Word not in index, try again')
        break
    
    

s
{'p': {'d': {'e': {'m': {'e': {'r': {'g': {'e': {'n': {'c': {'y': {'listWords': 'spdemergency'}}}}}}}}}}, 'e': {'e': {'c': {'h': {'listWords': 'speech'}}}}, 'i': {'d': {'e': {'r': {'listWords': 'spider'}}}}}}
p
{'d': {'e': {'m': {'e': {'r': {'g': {'e': {'n': {'c': {'y': {'listWords': 'spdemergency'}}}}}}}}}}, 'e': {'e': {'c': {'h': {'listWords': 'speech'}}}}, 'i': {'d': {'e': {'r': {'listWords': 'spider'}}}}}
i
{'d': {'e': {'r': {'listWords': 'spider'}}}}


In [72]:
st = "{'g': {'a': {'l': {'i': {'t': {'y': {'listWords': 'legality'}}}, 'l': {'y': {'listWords': 'legally'}}, 'listWords': 'legal'}}, 'i': {'s': {'l': {'a': {'t': {'i': {'o': {'n': {'listWords': 'legislation'}}}}}}}, 't': {'i': {'m': {'a': {'t': {'e': {'listWords': 'legitimate'}}}}}}}}, 'a': {'r': {'n': {'e': {'d': {'listWords': 'learned'}}, 'i': {'n': {'g': {'listWords': 'learning'}}}}}, 'v': {'e': {'listWords': 'leave'}}, 'd': {'i': {'n': {'g': {'listWords': 'leading'}}}}}, 'v': {'e': {'l': {'listWords': 'level'}}}, 'd': {'listWords': 'led'}}"

In [100]:
import re
w1 = re.findall("{'listWords[a-z: ']*'}",st)
temp_wor = []
for a in w1:
    w2 = re.findall(" '[a-z]*'",a.replace("![a-z]",""))
    w2 = ''.join(e for e in w2[0] if e.isalnum())
    temp_wor.append(w2)
    
return temp_wor

['legality', 'legally', 'legislation', 'legitimate', 'learned', 'learning', 'leave', 'leading', 'level', 'led']